In [2]:
from datetime import date, timedelta
import gc
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from tqdm.auto import tqdm
import lightgbm as lgb
import warnings
warnings.filterwarnings("ignore")

In [53]:
df_train = pd.read_csv('/home/centos/moon/ktw/favorita-grocery-sales-forecasting/data/train.csv/train.csv')
df_train.to_pickle("/home/centos/moon/ktw/favorita-grocery-sales-forecasting/data/df_train_favorita.pkl")
df_test = pd.read_csv("/home/centos/moon/ktw/favorita-grocery-sales-forecasting/data/test.csv/test.csv")
df_test.to_pickle("/home/centos/moon/ktw/favorita-grocery-sales-forecasting/data/df_test_favorita.pkl")
items = pd.read_csv("/home/centos/moon/ktw/favorita-grocery-sales-forecasting/data/items.csv/items.csv")
items.to_pickle("/home/centos/moon/ktw/favorita-grocery-sales-forecasting/data/items_favorita.pkl")
stores = pd.read_csv("/home/centos/moon/ktw/favorita-grocery-sales-forecasting/data/stores.csv/stores.csv")
stores.to_pickle("/home/centos/moon/ktw/favorita-grocery-sales-forecasting/data/stores_favorita.pkl")

In [54]:
%%time
input_dir = '/home/centos/moon/ktw/favorita-grocery-sales-forecasting/data/'

df_train = pd.read_pickle(input_dir + 'df_train_favorita.pkl')
df_train["date"] = pd.to_datetime(df_train["date"])
df_test = pd.read_pickle(input_dir + 'df_test_favorita.pkl')
df_test["date"] = pd.to_datetime(df_test["date"])
items = pd.read_pickle(input_dir + 'items_favorita.pkl')
stores = pd.read_pickle(input_dir + 'stores_favorita.pkl')

CPU times: user 25.6 s, sys: 14.4 s, total: 40 s
Wall time: 40 s


In [55]:
# 2017년 1월 1일보타 크거나 같은 날짜의 데이터들만 df_2017에 저장하고 df_train 제거
df_2017 = df_train.loc[df_train.date >= pd.datetime(2017,1,1)]
df_2017
#del df_train

,id,date,store_nbr,item_nbr,unit_sales,onpromotion
101688779,101688779,2017-01-01,25,99197,1.0,False
101688780,101688780,2017-01-01,25,103665,7.0,False
101688781,101688781,2017-01-01,25,105574,1.0,False
101688782,101688782,2017-01-01,25,105857,4.0,False
101688783,101688783,2017-01-01,25,106716,2.0,False
...,...,...,...,...,...,...
125497035,125497035,2017-08-15,54,2089339,4.0,False
125497036,125497036,2017-08-15,54,2106464,1.0,True
125497037,125497037,2017-08-15,54,2110456,192.0,False
125497038,125497038,2017-08-15,54,2113914,198.0,True


In [59]:
promo_2017_train = df_2017.set_index(
    ["store_nbr", "item_nbr", "date"])[["onpromotion"]].unstack(
        level=-1).fillna(False)
promo_2017_train.columns = promo_2017_train.columns.get_level_values(1)
promo_2017_train

date                2017-01-01  2017-01-02  2017-01-03  2017-01-04  \
store_nbr item_nbr                                                   
1         96995          False       False       False       False   
          99197          False       False       False       False   
          103520         False       False       False       False   
          103665         False       False       False       False   
          105574         False       False        True       False   
...                        ...         ...         ...         ...   
54        2109909        False       False       False       False   
          2110456        False       False       False       False   
          2113343        False       False       False       False   
          2113914        False       False       False       False   
          2116416        False       False       False       False   

date                2017-01-05  2017-01-06  2017-01-07  2017-01-08  \
store_nbr item_nbr                                                   
1         96995          False       False       False       False   
          99197          False       False       False       False   
          103520         False       False       False       False   
          103665         False       False       False       False   
          105574         False        True       False       False   
...                        ...         ...         ...         ...   
54        2109909        False       False       False       False   
          2110456        False       False       False       False   
          2113343        False       False       False       False   
          2113914        False       False       False       False   
          2116416        False       False       False       False   

date                2017-01-09  2017-01-10  ...  2017-08-06  2017-08-07  \
store_nbr item_nbr                          ...                           
1         96995          False       False  ...       False       False   
          99197          False       False  ...       False       False   
          103520         False       False  ...       False       False   
          103665         False       False  ...       False       False   
          105574         False       False  ...       False       False   
...                        ...         ...  ...         ...         ...   
54        2109909        False       False  ...       False       False   
          2110456        False       False  ...       False       False   
          2113343        False       False  ...       False       False   
          2113914        False       False  ...        True       False   
          2116416        False       False  ...       False        True   

date                2017-08-08  2017-08-09  2017-08-10  2017-08-11  \
store_nbr item_nbr                                                   
1         96995          False       False       False       False   
          99197          False       False       False       False   
          103520         False       False       False       False   
          103665         False       False       False       False   
          105574         False       False       False       False   
...                        ...         ...         ...         ...   
54        2109909        False       False       False       False   
          2110456        False       False       False       False   
          2113343        False       False       False       False   
          2113914         True        True        True       False   
          2116416        False       False       False       False   

date                2017-08-12  2017-08-13  2017-08-14  2017-08-15  
store_nbr item_nbr                                                  
1         96995          False       False       False       False  
          99197          False       False       False       False  
          10352

In [60]:
promo_2017_test = df_test.set_index(["store_nbr", "item_nbr", "date"])[["onpromotion"]].unstack(level=-1).fillna(False)
promo_2017_test.columns = promo_2017_test.columns.get_level_values(1)
promo_2017_test = promo_2017_test.reindex(promo_2017_train.index).fillna(False)
promo_2017 = pd.concat([promo_2017_train, promo_2017_test], axis=1)
promo_2017

date                2017-01-01  2017-01-02  2017-01-03  2017-01-04  \
store_nbr item_nbr                                                   
1         96995          False       False       False       False   
          99197          False       False       False       False   
          103520         False       False       False       False   
          103665         False       False       False       False   
          105574         False       False        True       False   
...                        ...         ...         ...         ...   
54        2109909        False       False       False       False   
          2110456        False       False       False       False   
          2113343        False       False       False       False   
          2113914        False       False       False       False   
          2116416        False       False       False       False   

date                2017-01-05  2017-01-06  2017-01-07  2017-01-08  \
store_nbr item_nbr                                                   
1         96995          False       False       False       False   
          99197          False       False       False       False   
          103520         False       False       False       False   
          103665         False       False       False       False   
          105574         False        True       False       False   
...                        ...         ...         ...         ...   
54        2109909        False       False       False       False   
          2110456        False       False       False       False   
          2113343        False       False       False       False   
          2113914        False       False       False       False   
          2116416        False       False       False       False   

date                2017-01-09  2017-01-10  ...  2017-08-22  2017-08-23  \
store_nbr item_nbr                          ...                           
1         96995          False       False  ...       False       False   
          99197          False       False  ...       False       False   
          103520         False       False  ...       False       False   
          103665         False       False  ...       False       False   
          105574         False       False  ...       False       False   
...                        ...         ...  ...         ...         ...   
54        2109909        False       False  ...       False       False   
          2110456        False       False  ...       False       False   
          2113343        False       False  ...       False       False   
          2113914        False       False  ...        True        True   
          2116416        False       False  ...       False       False   

date                2017-08-24  2017-08-25  2017-08-26  2017-08-27  \
store_nbr item_nbr                                                   
1         96995          False       False       False       False   
          99197          False       False       False       False   
          103520         False       False       False       False   
          103665         False       False       False       False   
          105574         False       False       False       False   
...                        ...         ...         ...         ...   
54        2109909        False       False       False       False   
          2110456        False       False       False        True   
          2113343        False       False       False       False   
          2113914         True        True        True        True   
          2116416        False       False       False       False   

date                2017-08-28  2017-08-29  2017-08-30  2017-08-31  
store_nbr item_nbr                                                  
1         96995          False       False       False       False  
          99197          False       False       False       False  
          10352

In [23]:
#del promo_2017_test, promo_2017_train

In [61]:
df_2017 = df_2017.set_index(
    ["store_nbr", "item_nbr", "date"])[["unit_sales"]].unstack(
        level=-1).fillna(0)
df_2017.columns = df_2017.columns.get_level_values(1)
df_2017


date                2017-01-01  2017-01-02  2017-01-03  2017-01-04  \
store_nbr item_nbr                                                   
1         96995            0.0         0.0         0.0         0.0   
          99197            0.0         0.0         3.0         1.0   
          103520           0.0         1.0         2.0         0.0   
          103665           0.0         0.0         0.0         3.0   
          105574           0.0         0.0         5.0        12.0   
...                        ...         ...         ...         ...   
54        2109909          0.0         0.0         0.0         0.0   
          2110456          0.0         0.0         0.0         0.0   
          2113343          0.0         0.0         0.0         0.0   
          2113914          0.0         0.0         0.0         0.0   
          2116416          0.0         0.0         0.0         0.0   

date                2017-01-05  2017-01-06  2017-01-07  2017-01-08  \
store_nbr item_nbr                                                   
1         96995            0.0         0.0         0.0         0.0   
          99197            1.0         1.0         2.0         0.0   
          103520           2.0         3.0         1.0         0.0   
          103665           2.0         2.0         1.0         2.0   
          105574           9.0         6.0         4.0         2.0   
...                        ...         ...         ...         ...   
54        2109909          0.0         0.0         0.0         0.0   
          2110456          0.0         0.0         0.0         0.0   
          2113343          0.0         0.0         0.0         0.0   
          2113914          0.0         0.0         0.0         0.0   
          2116416          0.0         0.0         0.0         0.0   

date                2017-01-09  2017-01-10  ...  2017-08-06  2017-08-07  \
store_nbr item_nbr                          ...                           
1         96995            0.0         0.0  ...         2.0         2.0   
          99197            0.0         1.0  ...         0.0         2.0   
          103520           1.0         1.0  ...         0.0         0.0   
          103665           0.0         7.0  ...         1.0         2.0   
          105574           3.0         9.0  ...         0.0         5.0   
...                        ...         ...  ...         ...         ...   
54        2109909          0.0         0.0  ...         1.0         1.0   
          2110456          0.0         0.0  ...      1343.0       652.0   
          2113343          0.0         0.0  ...         2.0         0.0   
          2113914          0.0         0.0  ...        17.0         0.0   
          2116416          0.0         0.0  ...         1.0         1.0   

date                2017-08-08  2017-08-09  2017-08-10  2017-08-11  \
store_nbr item_nbr                                                   
1         96995            0.0         0.0         1.0         0.0   
          99197            0.0         2.0         0.0         0.0   
          103520           3.0         0.0         3.0         1.0   
          103665           0.0         7.0         9.0         2.0   
          105574           7.0         6.0        10.0         5.0   
...                        ...         ...         ...         ...   
54        2109909          0.0         2.0         1.0         0.0   
          2110456        724.0        24.0         0.0         0.0   
          2113343          0.0         0.0         1.0         0.0   
          2113914         10.0        10.0         4.0         0.0   
          2116416          0.0         0.0         0.0         1.0   

date                2017-08-12  2017-08-13  2017-08-14  2017-08-15  
store_nbr item_nbr                                                  
1         96995            0.0         0.0         0.0         0.0  
          99197            0.0         0.0         0.0         0.0  
          10352

In [25]:
items = items.reindex(df_2017.index.get_level_values(1))

In [62]:
df_test

,id,date,store_nbr,item_nbr,onpromotion
0,125497040,2017-08-16,1,96995,False
1,125497041,2017-08-16,1,99197,False
2,125497042,2017-08-16,1,103501,False
3,125497043,2017-08-16,1,103520,False
4,125497044,2017-08-16,1,103665,False
...,...,...,...,...,...
3370459,128867499,2017-08-31,54,2132163,False
3370460,128867500,2017-08-31,54,2132318,False
3370461,128867501,2017-08-31,54,2132945,False
3370462,128867502,2017-08-31,54,2132957,False


In [29]:
def get_timespan(df, dt, minus, periods, freq='D'):
    return df[pd.date_range(dt - timedelta(days=minus), periods=periods, freq=freq)]

def prepare_dataset(t2017, is_train=True):
    X = pd.DataFrame({
        "day_1_2017": get_timespan(df_2017, t2017, 1, 1).values.ravel(),
        "mean_3_2017": get_timespan(df_2017, t2017, 3, 3).mean(axis=1).values,
        "mean_7_2017": get_timespan(df_2017, t2017, 7, 7).mean(axis=1).values,
        "mean_14_2017": get_timespan(df_2017, t2017, 14, 14).mean(axis=1).values,
        "mean_30_2017": get_timespan(df_2017, t2017, 30, 30).mean(axis=1).values,
        "mean_60_2017": get_timespan(df_2017, t2017, 60, 60).mean(axis=1).values,
        "mean_140_2017": get_timespan(df_2017, t2017, 140, 140).mean(axis=1).values,
        "promo_14_2017": get_timespan(promo_2017, t2017, 14, 14).sum(axis=1).values,
        "promo_60_2017": get_timespan(promo_2017, t2017, 60, 60).sum(axis=1).values,
        "promo_140_2017": get_timespan(promo_2017, t2017, 140, 140).sum(axis=1).values
    })
    for i in range(7):
        X['mean_4_dow{}_2017'.format(i)] = get_timespan(df_2017, t2017, 28-i, 4, freq='7D').mean(axis=1).values
        X['mean_20_dow{}_2017'.format(i)] = get_timespan(df_2017, t2017, 140-i, 20, freq='7D').mean(axis=1).values
    for i in range(16):
        X["promo_{}".format(i)] = promo_2017[f'{t2017 + timedelta(days=i)}'].values.astype(np.uint8)
    if is_train:
        y = df_2017[
            pd.date_range(t2017, periods=16)
        ].values
        return X, y
    return X

In [30]:
print("Preparing dataset...")
t2017 = date(2017, 5, 31)
X_l, y_l = [], []
for i in range(6):
    delta = timedelta(days=7 * i)
    X_tmp, y_tmp = prepare_dataset(t2017 + delta)
    X_l.append(X_tmp)
    y_l.append(y_tmp)
X_train = pd.concat(X_l, axis=0)
y_train = np.concatenate(y_l, axis=0)
del X_l, y_l
X_val, y_val = prepare_dataset(date(2017, 7, 26))
X_test = prepare_dataset(date(2017, 8, 16), is_train=False)

stores_items = pd.DataFrame(index=df_2017.index)
test_ids = df_test[['id']]

items = items.reindex( stores_items.index.get_level_values(1) )

Preparing dataset...


In [31]:
get_timespan(df_2017, date(2017, 7, 26), 1, 1)

date                2017-07-25
store_nbr item_nbr            
1         96995            0.0
          99197            0.0
          103520           2.0
          103665           3.0
          105574           5.0
...                        ...
54        2109909          0.0
          2110456         12.0
          2113343          0.0
          2113914         11.0
          2116416          0.0

[167515 rows x 1 columns]

In [71]:
print("Training and predicting models...")
params = {
    'num_leaves': 31,
    'objective': 'regression',
    'min_data_in_leaf': 200,
    'learning_rate': 0.02,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.7,
    'bagging_freq': 1,
    'metric': 'l2', # rmse
    'num_threads': 4,
    'device': 'cpu',
}

Training and predicting models...


In [68]:
X_train.shape, y_train.shape

((1005090, 40), (1005090, 16))

In [69]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [70]:
MAX_ROUNDS = 3000
val_pred = []
test_pred = []
cate_vars = []
for i in range(16):
    print("=" * 50)
    print("Step %d" % (i+1))
    print("=" * 50)
    dtrain = lgb.Dataset(
        X_train, label=y_train[:, i],
        categorical_feature=cate_vars,
        # perishable (썩기쉬운) 물품은 가중치를 1.5배 준다.
        weight=pd.concat([items["perishable"]] * 6) * 0.25 + 1
    )
    dval = lgb.Dataset(
        X_val, label=y_val[:, i], reference=dtrain,
        weight=items["perishable"] * 0.25 + 1,
        categorical_feature=cate_vars)
    bst = lgb.train(
        params, dtrain, num_boost_round=MAX_ROUNDS,
        valid_sets=[dtrain, dval], early_stopping_rounds=125, verbose_eval=500
    )
#     print("\n".join(("%s: %.2f" % x) for x in sorted(
#         zip(X_train.columns, bst.feature_importance("gain")),
#         key=lambda x: x[1], reverse=True
#     )))
    val_pred.append(bst.predict(
        X_val, num_iteration=bst.best_iteration or MAX_ROUNDS))
    test_pred.append(bst.predict(
        X_test, num_iteration=bst.best_iteration or MAX_ROUNDS))

print("Validation mse:", mean_squared_error(
    y_val, np.array(val_pred).transpose()))

lgb.plot_importance(bst,max_num_features=20)


Step 1


LightGBMError: Length of weights is not same with #data

In [21]:
print("Making submission...")
y_test = np.array(test_pred).transpose()
df_preds = pd.DataFrame(
    y_test, index=df_2017.index,
    columns=pd.date_range("2017-08-16", periods=16)
).stack().to_frame("unit_sales")
df_preds.index.set_names(["store_nbr", "item_nbr", "date"], inplace=True)

submission = df_test[["id"]].join(df_preds, how="left").fillna(0)
submission["unit_sales"] = np.clip(np.expm1(submission["unit_sales"]), 0, 1000)
submission.to_csv('lgb_max_3000.csv', float_format='%.4f', index=None)

Making submission...


NameError: name 'np' is not defined

In [22]:
submission

NameError: name 'submission' is not defined

# ensenble

In [2]:
import pandas as pd

In [50]:
first = pd.read_csv('./lstm_custum(small)_scaler_means.csv')

In [51]:
second = pd.read_csv('./lgb.csv')

In [52]:
first

,id,unit_sales
0,125497040,0.2188
1,125497041,0.4309
2,125497042,0.0000
3,125497043,1.2259
4,125497044,1.9041
...,...,...
3370459,128867499,0.0000
3370460,128867500,0.0000
3370461,128867501,0.0000
3370462,128867502,0.0000


In [53]:
second

,id,unit_sales
0,125497040,0.1876
1,125497041,0.2992
2,125497042,0.0000
3,125497043,0.9889
4,125497044,1.8322
...,...,...
3370459,128867499,0.0000
3370460,128867500,0.0000
3370461,128867501,0.0000
3370462,128867502,0.0000


In [54]:
ensenble = first

In [55]:
ensenble.unit_sales = round((0.6*first.unit_sales + 0.4*second.unit_sales), 4)

In [56]:
ensenble.to_csv('./ensenble_v1.csv', index= False)

In [57]:
ensenble

,id,unit_sales
0,125497040,0.2063
1,125497041,0.3782
2,125497042,0.0000
3,125497043,1.1311
4,125497044,1.8753
...,...,...
3370459,128867499,0.0000
3370460,128867500,0.0000
3370461,128867501,0.0000
3370462,128867502,0.0000
